# Description



#### Joint reconstruction of wideband single dish and interferometer data in CASA is [experimental](https://casa.nrao.edu/casadocs-devel/stable/casa-fundamentals/tasks-and-tools). Please use at own discretion.

The scope of parameters that has been
tested for CASA 5.7/6.1 can be found on the CASA Docs chapter page on
"[Joint Single Dish and Interferometer Image
Reconstruction](https://casa.nrao.edu/casadocs-devel/stable/imaging/image-combination/joint-sd-and-interferometer-image-reconstruction)"

&nbsp;

The
[**sdintimaging**](https://casa.nrao.edu/casadocs-devel/stable/global-task-list/)
task allows joint reconstruction of wideband single dish and
interferometer data.

Interferometer data are gridded into an
image cube (and corresponding PSF). The single dish image and PSF cubes
are combined with the interferometer cubes in a feathering step. The
joint image and PSF cubes then form inputs to any deconvolution
algorithm (in either *cube* or *mfs/mtmfs* modes). Model images from the
deconvolution algorithm are translated back to model image cubes prior
to subtraction from both the single dish image cube as well as the
interferometer data to form a new pair of residual image cubes to be
feathered in the next iteration. In the case of mosaic imaging, primary
beam corrections are performed per channel of the image cube, followed
by a multiplication by a common primary beam, prior to deconvolution.
Therefore, for mosaic imaging, this task always implements
*conjbeams=True* and *normtype=&rsquo;flatnoise&rsquo;*.

&nbsp;

![66b05f9d215777360fc1b1ce0147ce542eeb93b5.png](66b05f9d215777360fc1b1ce0147ce542eeb93b5.png)

&nbsp;

A more detailed description of the underlying algorithm, as well as
results from its testing, can be found on the CASA Docs chapter page on
"[Joint Single Dish and Interferometer Image
Reconstruction](https://casa.nrao.edu/casadocs-devel/stable/imaging/image-combination/joint-sd-and-interferometer-image-reconstruction)".&nbsp;
Note that the above diagram shows only the 'mtmfs' variant. Cube
deconvolution proceeds directly with the cubes in the green box above,
without the extra conversion back and forth to the multi-term basis.
Primary beam handling is also not shown in
this diagram, but full details (via pseudocode) are available in the
[reference
publication.](https://iopscience.iop.org/article/10.3847/1538-3881/ab1aa7)

&nbsp;



### Task Specification : sdintimaging

&nbsp;

The **sdintimaging** task shares a
significant number of parameters with the **tclean** task. In the
description below, parameters that are specific to sdintimaging are
listed with full details, but all others will reference the existing
tclean parameter documentation.

&nbsp;



#### Data Selection

-   All data selection options allowed for
    interferometer data. This set of parameters is identical to
    those in task **tclean**.

&nbsp;



#### Image Definition

-   Spatial dimensions are defined via the parameters :&nbsp;* imsize, cell, phasecenter,
    projection*

<!-- -->

-   Spectral dimensions for the major cycle are defined for cubes
    :&nbsp;* nchan,start, width, outframe,
    veltype, restfreq, interpolation  
    *

<!-- -->

-   Spectral dimensions for the minor cycle are chosen based on *specmode*.&nbsp; For
    *specmode='cube'* the minor cycle follows the same channelization as
    the major cycle. For *specmode='mfs'*, the choice of deconvolver and
    the setting of *'reffreq'* will decide the spectral coordinate
    system of the wideband image that is created after collapsing the
    cube images from the major cycle. For *deconvolver='mtmfs'* the
    appropriate cube-to-taylor (and reverse) conversions are applied.



##### Specifying Both Cube and MFS settings (for *specmode='mfs'*) :

In **sdintimaging**, with MFS deconvolution, one *needs to specify both
cube and mfs settings for frequency coordinates because in this usage
mode the major cycle is done with cubes and the minor cycle with mfs*.
This detail is different from the tclean task.&nbsp; A few general rules
to follow for a MFS (or MTMFS) minor cycle are

-   The reffreq must lie between the ends of the cube frequency range
    (default based on data selection, or explicitly specified using
    *start, width, nchan*).&nbsp; If this is not the case, an error
    message will appear.&nbsp; If left at its default value of
    *reffreq=''*, it will be automatically computed to be the mean of
    the first and last channel frequencies and a log message is printed
    with the new value.

<!-- -->

-   For a wideband imaging run with nterms, at least nterms channels
    must be present in the input cubes. Ideally, in order to fully
    capture spectral variations and also guard against missing data, it
    is recommended that the cube be defined with between 5 and 20
    channels. More may of course be used, especially in order to avoid
    bandwidth smearing within channels, but a larger number of channels
    will cause the feathering step to take longer.&nbsp; Warnings are
    printed if the nchan of the cube is small (less than 5) or too large
    (more than 50), and the task will exit with an error if nchan \<
    nterms.

<!-- -->

-   The sdintimaging task will perform the above checks on the input
    parameters and report problems/warnings as appropriate.&nbsp;&nbsp;
    The internal automation of some of these settings is on our 'Future
    Work' list.

&nbsp;



#### Single Dish data input&nbsp;

&nbsp;

-   Image cubes that represent the observed SD image per channel and the
    corresponding SD beam :&nbsp; *sdimage, sdpsf*

<!-- -->

-   Both the sdimage and sdpsf image cubes must contain per plane
    restoringbeams that represent the effective SD beam.&nbsp; Per-plane
    restoring beams may be added to an existing image cube using  *ia.open()*, a loop over channels with
    *ia.setrestoringbeam(..)*, and *ia.close()*

<!-- -->

-   Ideally, the imsize, cellsize, and phasecenter of the SD cube should
    match that of the INT cubes specified by imsize, cellsize,
    phasecenter.&nbsp;&nbsp; However in case of a detected mismatch, the
    *ia.regrid()* method is called internally to convert it to the
    target csys prior to continuing. It is expected that such a regrid
    is possible and in case of error, the user should see a warning and
    suggestion to experiment with the imregrid task to reformat their
    input SD cube.

<!-- -->

-   The frequency axis of the SD cubes must exactly match the INT cube
    spectral axis defined by nchan, start, width.&nbsp; Note that in the
    internal imregrid call, the frequency axis is not regridded. *This
    means that nchan, start and width specified in the task interface
    must match the frequency coordinates of the input SD image.*
    -   Use a helper method (shown in the
        [ALMA M100
        example](https://casa.nrao.edu/casadocs-devel/stable/global-task-list/task_sdintimaging/examples)
        below) to extract nchan/start/width parameters from the SD Image
        cube, and supply these as inputs to sdintimaging to exactly
        match the frequency coordinates of the SD and INT cubes. 

<!-- -->

-   The order of the direction, stokes, and spectral axes must match the
    INT cubes, typically RA,DEC,Stokes,Channel

<!-- -->

-   Blank channels (sum of pixel amplitudes=0) are internally flagged
    and left out of the joint reconstruction.&nbsp;&nbsp; So, one way to
    tell the algorithm to ignore some channels in the input SD cube is
    to force all pixel values to zero.

<!-- -->

-   A convenience option has been provided within sdintimaging to
    auto-generate simple SD PSF cubes. If sdpsf='', a PSF cube is
    calculated by evaluating Gaussians based on the restoringbeam
    information per channel read from the input SD Image cube.&nbsp;
    This option is useful if only an SD Image cube is available as the
    output of the single dish imaging step.

Please see the [ALMA M100
example](https://casa.nrao.edu/casadocs-devel/stable/global-task-list/task_sdintimaging/examples) section for sample code and task calls that
illustrates the simplest way of setting up these inputs.&nbsp; 

&nbsp;

To use SD PSFs that represent actual SD beam
patterns, please read the following details. 

-   The SD PSF must contain a model of the single dish beam at the same
    world-coordinate location as the imaging phasecenter that is
    specified (or assumed via the supplied MS, when
    *phasecenter=&rsquo;&rsquo;*), it must be normalized to peak 1, and
    the PSF cube must contain corresponding restoring beams per channel.

<!-- -->

-   It is also expected that the single dish PSF peak is at the image
    center after regridding (same as the interferometer PSF). An
    internal check will look for position shifts (subpixel shifts too)
    and if offsets are 0.001 of a degree or more, it will not
    proceed.&nbsp; A way around this is to manually re-evaluate the SD
    PSF directly onto the coordinate system of one of the intermediate
    INT images such that the middle pixel contains the peak of the PSF.
    An alternative is to use the *sdpsf=''* option, with which one can
    approximate the SD PSF.

<!-- -->

-   Other ideas to create an SD PSF : Use the SD image cube for header
    information and cube dimensions. Create an empty CASA image, fill it
    with evaluated Gaussians that match the SD beam size per channel. A
    sample script is provided
    [here](https://github.com/urvashirau/WidebandSDINT/blob/master/ScriptForRealData/make_gauss_beam_cube.txt).

<!-- -->

-   The SD PSFs (in this case for the simulated examples/tests) are
    typically generated by calculating disk-shaped aperture functions of
    the appropriate dish diameter, taking a Fourier transform and
    squaring and normalizing the result.

&nbsp;



#### Data Combination options

The sdintimaging task may be run in three data combination modes via
the* usedata* parameter.&nbsp;

-   **'sdint' :**&nbsp; Use the
    interferometer and single dish data in a joint reconstruction.&nbsp;
    Specification of the *&lsquo;sdgain&rsquo;* and
    &lsquo;*dishdia&rsquo;* are the same as for the **feather** task.
    The method in the feather task is called internally to combine image
    cubes and PSF cubes prior to deconvolution.
    -   For *specmode='mfs'*, each channel is pb-corrected to flat sky
        and then a common primary beam (and mask) is applied prior to
        deconvolution. The common PB is computed as a weighted average
        of PBs, using the .sumwt per channel.&nbsp;
    -   When the INT or the SD cubes contain flagged (i.e. empty)
        channels, they are left out of the joint reconstruction.
        Therefore, only those channels that have both INT and SD images,
        are used.

<!-- -->

-   '**sd**' : Use only the single-dish data and enable deconvolution of
    the single dish image cubes. Both cube and wideband multi-term
    deconvolution of single dish data are possible. Note that this mode
    (currently) still requires an interferometer MS to be supplied in
    order to construct image templates. This option is experimental and
    has passed only the tests reported in the publication and the
    examples shown in CASAdocs.

<!-- -->

-   **'int'** : Uses only interferometer data. For gridder=*'mosaic'*
    and *'awproject'*, it implements a wideband mosaic scheme similar to
    those offered via task tclean, but with the concept of conjugate-pb
    correction implemented in the image domain. It does so by taking a
    flat-sky normalization per channel, followed by a flat-noise
    rescaling to apply a common primary beam to all channels, and
    subsequently collapsing into taylor images for deconvolution. This
    option is experimental and has passed only the most basic tests.
    Further characterization and comparison to the equivalent imaging
    modes in tclean will be done after the CASA 6.1 release.&nbsp;
    Therefore, please use only with caution.



##### &nbsp;



##### Tuning the sdgain parameter :

The *sdgain* parameter acts as an
image weighting option by being applied both to the data as well as the
PSFs during combination. Setting values away from 1.0 adjusts the
relative weight of the SD information to be higher than INT cube,
separately for each channel. Initial demonstrations have shown promise,
but the robustness of this algorithm control will become clearer with
more practical use.

&nbsp;

-   A high sdgain value ( \> 1.0 ) has been demonstrated to emphasize
    extended emission without changing the high resolution structure
    (see the ALMA M100 example in the "[Joint Single Dish and
    Interferometer Image
    Reconstruction](https://casa.nrao.edu/casadocs-devel/stable/imaging/image-combination/joint-sd-and-interferometer-image-reconstruction)"
    page).&nbsp;&nbsp; However, when using a high sdgain, please
    remember to monitor the shape of the joint PSF to look for signs of
    angular resolution loss due to weighting the SD data much too
    high.&nbsp;

<!-- -->

-   A low sdgain value ( \< 1.0 ) has also been shown to be useful in
    reducing the effect of the usually high SD noise in the joint
    reconstruction while still preserving flux correctness (see the
    [algorithm
    publication](https://iopscience.iop.org/article/10.3847/1538-3881/ab1aa7/meta))
    .&nbsp; This mode could be useful when the SD image signal-to-noise
    ratio is high enough to match that of the interferometer images,
    even if the rms noise of the SD data is higher than the INT image
    rms (which can happen when the flux of the SD data is higher than
    that of the INT data).

&nbsp;

&nbsp;



#### Imaging and Deconvolution Options

Parameters that control interferometer-gridding/imaging and
deconvolution options are *specmode, gridder, deconvolver* (and
associated sub-parameters similar to **tclean**).

-   **Specmode** : Supported modes include&nbsp; *specmode='cube'&nbsp;*
    with any single-term deconvolver, and&nbsp; *specmode='mfs'* with
    any deconvolver (including multi-term). These options represent
    different conversion routines between the feathered cubes and the
    inputs/outputs for deconvolution.
    -   *&lsquo;cube&rsquo;*: the cubes are sent as is to the
        deconvolver and the output model cube is directly passed to the
        major cycle.
    -   *&lsquo;mfs&rsquo;*: the cubes are averaged to form a continuum
        image and continuum PSF prior to deconvolution and the model
        image is expanded out to an image model cube prior to the next
        major cycle.
    -   *&lsquo;mtmfs&rsquo;*: the cubes are converted to
        Taylor-weighted averages in accordance with the MTMFS algorithm
        and the model Taylor coefficient image output from the
        deconvolver are evaluated back onto a model image cube prior to
        the major cycle. This image reshaping follows the diagram at the
        top of this page.

All frequency averages in the Cube to Taylor conversions and in the
calculation of a common Primary Beam use the interferometer
sum-or-weight spectrum as frequency-dependent weights, multiplied by a
1-0 flag to identify channels with valid images in both the SD and INT
cubes

-   **Deconvolvers** : Algorithms supported are *&lsquo;multiscale',
    'hogbom&rsquo;* and *'clark'* for *cube* and *mfs(nterms=1)* imaging
    and *&lsquo;mtmfs&rsquo;* for multi-term mfs imaging. However, for
    use cases where single dish data are required along with
    interferometer data, multiscale deconvolution is most appropriate to
    get accurate reconstructions at multiple spatial scales. The
    *&lsquo;multiscale&rsquo;* deconvolver applies to
    *specmode=&rsquo;cube&rsquo;* and *'mfs(nterms=1)&rsquo;* and the
    *&lsquo;mtmfs&rsquo;* deconvolver applies to the
    *specmode=&rsquo;mfs(nterms\>1)&rsquo;*. In all cases, the
    *&lsquo;scales&rsquo;* parameter is also relevant as it sets the
    list of scale sizes to use during deconvolution.The
    *&lsquo;hogbom&rsquo;* deconvolver is relevant only when used with
    *usedata=&rsquo;sdonly&rsquo;* to deconvolve unresolved sources.

<!-- -->

-   **Gridders** :&nbsp; Any gridder supported by task tclean may be
    used with **sdintimaging**. Two options that represent different
    normalization schemes are *'standard'* and *'mosaic'* (or
    *'awproject'*). Similar to tclean, the&nbsp;
    *&lsquo;standard&rsquo;* gridder does not consider primary beams and
    represents one mode of operation that is valid only in the central
    region of the interferometer primary beam. The
    *&lsquo;mosaic&rsquo;* and *'awproject'* gridders account for
    primary beams and are appropriate for full-beam or joint mosaic
    images.&nbsp; For these two A-Projection gridders, the normtype is
    always *'flatnoise'* and conjbeams is implemented via an
    image-domain scheme not offered by task tclean.&nbsp; Note also that
    although the *&lsquo;awproject&rsquo;* gridder may be used
    interchangeably with *&lsquo;mosaic&rsquo;*, this mode will not be
    tested for the initial release of this task (CASA 5.7/6.1).

&nbsp;



#### Iteration Control&nbsp; and Automasking

Iteration contol and automasking parameters are identical to those used in task
tclean, with the same rules and conventions applied to stopping
criteria.

&nbsp;



#### Output Images

The initial version of the sdintimaging task produces many intermediate
images which persist after the end of the task.&nbsp; The naming
convention of the images is more complex than the tclean task.

<table>
<colgroup>
<col style="width: 50%" />
<col style="width: 50%" />
</colgroup>
<tbody>
<tr class="odd">
<td><p>&lt;imagename&gt;.sd.cube.{image,psf}</p>
<p>&lt;imagename&gt;.sd.cube.{model,residual}</p></td>
<td><p>Image cubes onto which the input Single Dish image and psf cubes are regridded.</p>
<p>Intermediate products containing the model image cube that is subtracted from the SD image to make the SD residual</p></td>
</tr>
<tr class="even">
<td><p>&lt;imagename&gt;.int.cube.{residual, psf, sumwt,weight,pb)</p>
<p>&lt;imagename&gt;.int.cube.{model}</p>
<p>&#xA0;</p></td>
<td><p>Image cubes made from only the interferometer data</p>
<p>Intermediate product. Cube model image used for model prediction and residual calculation.</p></td>
</tr>
<tr class="odd">
<td><p>&lt;imagename&gt;.joint.cube.{residual, psf}</p>
<p>&lt;imagename&gt;.joint.multiterm.{residual,psf}.{tt0,tt1[,tt2]}</p></td>
<td><p>Feathered cubes for the residual and psf.&#xA0;&#xA0; For cube minor cycles, these are also the inputs to the deconvolver.</p>
<p>Multi-term residual images and spectral PSFs constructed from the above feathered cubes. These are inputs to the minor cycle for multi-term deconvolution</p></td>
</tr>
<tr class="even">
<td><p>&lt;imagename&gt;.joint.cube.{image, sumwt, weight, pb,model, mask,pbcor}</p></td>
<td><p>For cube minor cycles, all standard data products</p></td>
</tr>
<tr class="odd">
<td><p>&lt;imagename&gt;.joint.multiterm.{image, sumwt, weight, pb, model, mask, alpha,pbcor}&#xA0; with&#xA0; {.tt0, .tt1, .tt2 } extensions as appropriate.</p></td>
<td>For multi-term minor cycles, all standard data products</td>
</tr>
</tbody>
</table>

&nbsp;This long list of output and intermediate images is likely to be
pruned in a future release.

&nbsp;

&nbsp;

For more information and examples on the
functionality of the sdintimaging task, see the CASA Docs chapter page
on "[Joint Single Dish and Interferometer Image
Reconstruction](https://casa.nrao.edu/casadocs-devel/stable/imaging/image-combination/joint-sd-and-interferometer-image-reconstruction)"

&nbsp;